<center>

# **如何运行Heudiconv**

</center>

***

**前提条件**

1. 已安装 heudiconv docker 和 dcm2niix。
2. 准备好待处理的 DICOM 数据集（可参考 quickstart 数据集）。
3. 确保 DICOM 文件未压缩（heudiconv 不支持压缩 DICOM）。

注：进行docker pull 不成功，并且是因为网络问题的时候，可参考以下链接：

- https://gitee.com/wanfeng789/docker-hub#docker-hub
- https://www.bilibili.com/video/BV1Zn4y19743/ 
- https://status.1panel.top/status/docker